In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import requests
from bs4 import BeautifulSoup

def get_city_info_wikipedia(city_name):
    city_information=""
    # Passo 1: Primeiro, fazemos uma consulta para obter o ID da página da cidade
    url = f'https://pt.wikipedia.org/w/api.php?action=query&format=json&titles={city_name}'

    response = requests.get(url)
    data = response.json()

    # Extraindo o ID da página da resposta
    pages = data['query']['pages']
    page_id = next(iter(pages))  # Pegando o primeiro ID da página (único)
    
    # Passo 2: Consultar o conteúdo da página usando o ID obtido
    if page_id != '-1':  # Se a página existir
        city_information += f"Informações sobre a cidade: {city_name}\n"
        page_url = f'https://pt.wikipedia.org/wiki/{city_name.replace(" ", "_")}'
        city_information += f"Link para mais detalhes: {page_url}\n\n"
        
        # Requisição para pegar o conteúdo da página
        url_details = f'https://pt.wikipedia.org/w/api.php?action=parse&format=json&pageid={page_id}&prop=text&formatversion=2'
        response_details = requests.get(url_details)
        data_details = response_details.json()

        # Pegando o conteúdo (HTML da página)
        content = data_details['parse']['text']
        
        # Usando BeautifulSoup para parsear o HTML
        soup = BeautifulSoup(content, 'html.parser')

        # Exemplo de busca por informações específicas de Clima, Economia, Educação, Transporte, População
        # Buscando essas informações com base nas seções encontradas

        city_information += f"Informações encontradas sobre:{city_name}\n"
        
        # Buscando seções de interesse
        for section in soup.find_all(['h2', 'h3', 'h4']):
            section_text = section.get_text(strip=True).lower()
    
            if any(keyword in section_text for keyword in ['clima', 'economia', 'educação', 'transporte', 'população','lazer','bairros','gastronomia',
                                                           'parques e jardins','demografia','turismo','saude','desporto']):
                city_information += f"\n{section_text.capitalize()}:\n"
                
                # Pegando o conteúdo da seção e exibindo
                section_next = section.find_next(['p', 'ul'])
                if section_next:
                    city_information += f"- {section_next.get_text(strip=True)}\n"
        
 # Exemplo: Extrair tabelas (como infoboxes)
        city_information += "\nInfobox encontrados:"
        infobox = soup.find('table', {'class': 'infobox'})
        if infobox:
            # Para cada linha da tabela do infobox
            for row in infobox.find_all('tr'):
                # Encontrando os <td> de cada linha (dados e valores)
                cells = row.find_all('td')
                if len(cells) == 2:  # A linha tem 2 células (título e valor)
                    title = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)

                    # Exibe o título e o valor extraído
                    city_information += f"- {title}: {value}\n"
                    
                    # Caso haja links, podemos mostrar o URL
                    
                    # link = cells[1].find('a')
                    # if link:
                    #     link_url = link.get('href')
                    #     print(f"  Link: {link_url}")

    else:
        city_information += f"Página '{city_name}' não encontrada."

    return city_information

In [9]:
# read cities by file csv
def read_cities_from_file(file_path):
    import pandas as pd
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        cities = df['Cidades'].tolist()  # Supondo que a coluna com os nomes das cidades seja 'Nome'
        return cities
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return []

In [10]:
filepath = '..\\..\\Bases\\Municipios\\' + 'portugalMunicipios' + '.csv'
cities = read_cities_from_file(filepath)

In [12]:
# Exemplo de uso:
for city_name in cities:
    print(f"Buscando informações para a cidade: {city_name}")
    city_info = get_city_info_wikipedia(city_name)

    filepath = '..\\..\\Bases\\Wikipedia\\' + city_name + '.txt'
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(city_info)   


Buscando informações para a cidade: Arcos de Valdevez
Buscando informações para a cidade: Caminha
Buscando informações para a cidade: Melgaço
Buscando informações para a cidade: Monção
Buscando informações para a cidade: Paredes de Coura
Buscando informações para a cidade: Ponte da Barca
Buscando informações para a cidade: Ponte de Lima
Buscando informações para a cidade: Valença
Buscando informações para a cidade: Viana do Castelo
Buscando informações para a cidade: Vila Nova de Cerveira
Buscando informações para a cidade: Amares
Buscando informações para a cidade: Barcelos
Buscando informações para a cidade: Braga
Buscando informações para a cidade: Esposende
Buscando informações para a cidade: Terras de Bouro
Buscando informações para a cidade: Vila Verde
Buscando informações para a cidade: Cabeceiras de Basto
Buscando informações para a cidade: Fafe
Buscando informações para a cidade: Guimarães
Buscando informações para a cidade: Mondim de Basto
Buscando informações para a cidade: 